In [ ]:
import tarfile
import urllib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import plotting
from scipy import stats
from sklearn import base
from sklearn import compose, impute, pipeline, preprocessing
from sklearn import linear_model, tree, ensemble, svm
from sklearn import metrics, model_selection
from sklearn.externals import joblib

In [ ]:
%matplotlib inline

This notebook contains my solutions to Chapter 2 of _Hands-on Machine Learning with Scikit-learn and TensorFlow_. Author's solutions are also avalable on GitHub.

* https://github.com/ageron/handson-ml/blob/master/02_end_to_end_machine_learning_project.ipynb

Major difference between my solutions and the authors is that my solutions take advantage of some newer features of the Scikit-Learn [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) module and the new [compose](https://scikit-learn.org/stable/modules/compose.html#combining-estimators) module for composing analysis pipelines.

# Get the Data

In general you want to automate as much of the process of accessing the data as possible in order to make it easier to get fresh data as it becomes available!

In [ ]:
def fetch_housing_data(datasets_url):
    path, _ = urllib.request.urlretrieve(f"{datasets_url}/housing/housing.tgz",
                                         "../data/housing/housing.tgz")
    with tarfile.open(path) as tf:
        tf.extractall("../data/housing/")

In [ ]:
_datasets_url = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets"
fetch_housing_data(_datasets_url)

In [ ]:
_dtype = {"ocean_proximity": "category"}

housing_df = pd.read_csv("../data/housing/housing.csv",
                         dtype=_dtype)

## Take a Quick Look at the Data Structure

In [ ]:
housing_df.head()

In [ ]:
housing_df.tail()

In [ ]:
housing_df.info()

Flat text files (such as CSV files) are inefficient storage formats.

* Flat text files are schema-less: data-type information is not stored together with the data so it must be re-coded everytime the raw data is loaded.
* Flat text files can not be efficiently stored.
* Flat text files can not be efficiently queried.

Always a good idea to convert raw text files to efficienty storage formats such as [Apache Parquet](https://parquet.apache.org/) first and then read/query the data from the resulting parquet files.

In [ ]:
housing_df.to_parquet("../data/housing/housing.parquet", index=False)

In [ ]:
import pyarrow.parquet as pq

_table = (pq.ParquetFile("../data/housing/housing.parquet")
            .read(use_pandas_metadata=True))

housing_df = _table.to_pandas(strings_to_categorical=True)

In [ ]:
housing_df.info()

In [ ]:
housing_df.describe()

In [ ]:
(housing_df.get("ocean_proximity")
           .value_counts())

In [ ]:
_ = housing_df.hist(bins=50, figsize=(20, 15))

## Create a Test Set

In [ ]:
preprocessing.KBinsDiscretizer?

In [ ]:
model_selection.train_test_split?

In [ ]:
_prng = np.random.RandomState(42)

# discretize a continuous variable and use result for stratified sampling!
_discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode="ordinal")
_stratify = _discretizer.fit_transform(housing_df.loc[:, ["median_income"]])

training_df, testing_df = model_selection.train_test_split(housing_df,
                                                           test_size=0.20,
                                                           stratify= _stratify,
                                                           random_state=_prng)

In [ ]:
training_df.head()

In [ ]:
training_df.info()

In [ ]:
testing_df.head()

In [ ]:
testing_df.info()

In [ ]:
_ = joblib.dump(training_df, "../data/housing/training.pkl")
_ = joblib.dump(testing_df, "../data/housing/testing.pkl")

# Discover and Visualize the Data to Gain Insights

In [ ]:
training_df = joblib.load("../data/housing/training.pkl")

In [ ]:
training_df.head()

## Visualizing Geographic Data

In [ ]:
_fig, _ax = plt.subplots(1, 1, figsize=(10, 7))

_marker_sizes = (training_df.loc[:, "population"]
                            .div(100))

_cmap = plt.get_cmap("viridis")

_kwargs = {'c': "median_house_value",
           's': _marker_sizes,
           "label": "population",
           "alpha": 0.4,
           "cmap": _cmap,
           "ax": _ax}

_ = (training_df.plot
                .scatter(x="longitude", y="latitude", **_kwargs))

## Looking for Correlations

In [ ]:
(training_df.corr()
            .loc[:, "median_house_value"]
            .sort_values(ascending=False))

In [ ]:
_attributes = ["median_house_value",
               "median_income",
               "total_rooms",
               "housing_median_age"]

_ = plotting.scatter_matrix(training_df.loc[:, _attributes], figsize=(12, 8))

In [ ]:
_ = (training_df.plot
                .scatter(x="median_income", y="median_house_value", alpha=0.1))

## Experimenting with Attribute Combinations

In [ ]:
class HousingFeatureCreator(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        
        # examples of hand-crafted features
        _rooms_per_household = (df.loc[:, "total_rooms"]
                                  .div(df.loc[:, "households"]))
        _bedrooms_per_room = (df.loc[:, "total_bedrooms"]
                                .div(df.loc[:, "total_rooms"]))
        _population_per_household = (df.loc[:, "population"]
                                       .div(df.loc[:,"households"]))

        _features = {"rooms_per_household": _rooms_per_household,
                     "bedrooms_per_room": _bedrooms_per_room,
                     "population_per_household": _population_per_household}

        return df.assign(**_features)

In [ ]:
feature_creator = HousingFeatureCreator()
transformed_features_df = feature_creator.fit_transform(training_df)

In [ ]:
(transformed_features_df.corr()
                        .loc[:, "median_house_value"]
                        .sort_values(ascending=False))

# Prepare the Data for ML Algorithms

## Data Cleaning

In [ ]:
(training_df.isna()
            .sum())

In [ ]:
impute.SimpleImputer?

## Handling Text and Categorical Attributes

In [ ]:
preprocessing.OneHotEncoder?

In [ ]:
preprocessing.OrdinalEncoder?

## Feature Scaling

In [ ]:
preprocessing.StandardScaler?

## Transformation Pipelines

In [ ]:
pipeline.make_pipeline?

In [ ]:
numeric_features_transformer = pipeline.make_pipeline(
    impute.SimpleImputer(strategy="median"),
    preprocessing.StandardScaler()
)

In [ ]:
categorical_features_transformer = pipeline.make_pipeline(
    impute.SimpleImputer(strategy="most_frequent"),
    preprocessing.OneHotEncoder()
)

In [ ]:
compose.make_column_transformer?

In [ ]:
_numerical_columns = ["longitude",
                      "latitude",
                      "housing_median_age",
                      "total_rooms",
                      "total_bedrooms",
                      "population",
                      "households",
                      "median_income",
                      "rooms_per_household",
                      "bedrooms_per_room",
                      "population_per_household"]

_categorical_columns = ["ocean_proximity"]

column_transformer = compose.make_column_transformer(
    (numeric_features_transformer, _numerical_columns),
    (categorical_features_transformer, _categorical_columns),
    remainder="drop",
)

In [ ]:
preprocessing_pipeline = pipeline.make_pipeline(
    feature_creator,
    column_transformer
)

In [ ]:
training_features_arr = preprocessing_pipeline.fit_transform(training_df)

In [ ]:
# 11 numeric features + 5 one-hot encoded categorical feature = 16 cols!
training_features_arr.shape

In [ ]:
training_target_arr = (training_df.loc[:, "median_house_value"]
                                  .values)

In [ ]:
training_target_arr.shape

# Select and Train a Model

## Training and Evaluating on the Training Set

### Linear Regression

See the Scikit-learn [docs](https://scikit-learn.org/stable/modules/linear_model.html) on Linear Regression for more details about this regression technique.

In [ ]:
linear_regressor = linear_model.LinearRegression(fit_intercept=False)
linear_regressor.fit(training_features_arr, training_target_arr)

In [ ]:
linear_regressor_predictions = linear_regressor.predict(training_features_arr)

In [ ]:
linear_regressor_predictions

In [ ]:
training_target_arr

In [ ]:
linear_regressor_mse = metrics.mean_squared_error(training_target_arr, linear_regressor_predictions)
linear_regressor_rmse = linear_regressor_mse**0.5

In [ ]:
# error units are USD!
linear_regressor_rmse

In [ ]:
_ = joblib.dump(linear_regressor, "../models/housing/linear-regressor.pkl")

### Decision Trees

See Scikit-learn [docs](https://scikit-learn.org/stable/modules/tree.html) on decision trees for the details of this regression technique.

In [ ]:
decision_tree_regressor = tree.DecisionTreeRegressor()
decision_tree_regressor.fit(training_features_arr, training_target_arr)

In [ ]:
decision_tree_regressor_predictions = decision_tree_regressor.predict(training_features_arr)
decision_tree_regressor_mse = metrics.mean_squared_error(training_target_arr, decision_tree_regressor_predictions)
decision_tree_regressor_rmse = decision_tree_regressor_mse**0.5

In [ ]:
decision_tree_regressor_rmse

In [ ]:
_ = joblib.dump(decision_tree_regressor, "../models/housing/decision-tree-regressor.pkl")

### Random Forests

See the Scikit-learn [docs](https://scikit-learn.org/stable/modules/ensemble.html) on Random Forests for more information about this regression technique.

In [ ]:
random_forest_regressor = ensemble.RandomForestRegressor()
random_forest_regressor.fit(training_features_arr, training_target_arr)

In [ ]:
def compute_rmse(regressor):
    _predictions = regressor.predict(training_features_arr)
    _mse = metrics.mean_squared_error(training_target_arr, _predictions)
    rmse = _mse**0.5
    return rmse

In [ ]:
compute_rmse(random_forest_regressor)

In [ ]:
_ = joblib.dump(random_forest_regressor, "../models/housing/random-forest-regressor.pkl")

### Gradient Boosted Trees

See the Scikit-learn [docs](https://scikit-learn.org/stable/modules/ensemble.html) on Gradient Boosted Trees for more information about this regression technique.

In [ ]:
gradient_boosting_regressor = ensemble.GradientBoostingRegressor()
gradient_boosting_regressor.fit(training_features_arr, training_target_arr)

In [ ]:
compute_rmse(gradient_boosting_regressor)

In [ ]:
_ = joblib.dump(gradient_boosting_regressor, "../models/housing/gradient-boosting-regressor.pkl")

## Better Evaluation using Cross-Validation

In [ ]:
NUMBER_CV_FOLDS = 5
NUMBER_CV_JOBS = 1 # NUMBER_CV_FOLDS
VERBOSITY = 10

In [ ]:
regressors = [linear_regressor,
              decision_tree_regressor,
              random_forest_regressor,
              gradient_boosting_regressor]

cv_results = {}
for regressor in regressors:
    scores = model_selection.cross_val_score(regressor, 
                                             X=training_features_arr,
                                             y=training_target_arr,
                                             scoring="neg_mean_squared_error",
                                             cv=NUMBER_CV_FOLDS,
                                             n_jobs= NUMBER_CV_JOBS,
                                             verbose=VERBOSITY)
    rmses = np.sqrt(-scores)
    cv_results[regressor] = rmses
    

In [ ]:
def display_cv_results(cv_results):
    for regressor, rmses in cv_results.items():
        name = type(regressor).__name__
        print(f"Regressor: {name}\n\tAverage RMSE: {np.mean(rmses)}\n\tStandard Deviation RMSE: {np.std(rmses)}\n")

In [ ]:
display_cv_results(cv_results)

# Fine-Tune Your Model

## Grid Search

In [ ]:
model_selection.GridSearchCV?

In [ ]:
_param_grid = [
    {"n_estimators": [3, 10, 30], "max_features": [2, 4, 6, 8]},
    {"bootstrap": [False], "n_estimators": [3, 10], "max_features": [2, 3, 4]}
]

random_forest_grid_search = model_selection.GridSearchCV(random_forest_regressor,
                                                         param_grid=_param_grid,
                                                         scoring="neg_mean_squared_error",
                                                         cv=NUMBER_CV_FOLDS,
                                                         n_jobs=NUMBER_CV_JOBS,
                                                         verbose=VERBOSITY)

random_forest_grid_search.fit(training_features_arr, training_target_arr)

In [ ]:
_ = joblib.dump(random_forest_grid_search, "../models/housing/random-forest-grid-search.pkl")

In [ ]:
(-random_forest_grid_search.best_score_)**0.5

## Randomized Search

In [ ]:
model_selection.RandomizedSearchCV?

In [ ]:
ensemble.GradientBoostingRegressor?

In [ ]:
_param_distributions = {
    "n_estimators": stats.geom(p=0.01),
     "min_samples_split": stats.beta(a=1, b=99),
     "min_samples_leaf": stats.beta(a=1, b=999),
}

gradient_boosting_randomized_search = model_selection.RandomizedSearchCV(
    gradient_boosting_regressor,
    param_distributions=_param_distributions,
    scoring="neg_mean_squared_error",
    n_iter=10,
    cv=NUMBER_CV_FOLDS,
    n_jobs=NUMBER_CV_JOBS,
    verbose=VERBOSITY
)

gradient_boosting_randomized_search.fit(training_features_arr, training_target_arr)

In [ ]:
(-gradient_boosting_randomized_search.best_score_)**0.5

In [ ]:
_ = joblib.dump(gradient_boosting_randomized_search, "../models/housing/gradient-boosting-randomized-search.pkl")

## Analyze the Best Models and Their Errors

In [ ]:
(random_forest_grid_search.best_estimator_
                          .feature_importances_)

In [ ]:
_ocean_proximity_categories = (training_df.loc[:, "ocean_proximity"]
                                          .cat
                                          .categories)
_features_names = (_numerical_columns + list(_ocean_proximity_categories))

_feature_importances = (random_forest_grid_search.best_estimator_
                                                 .feature_importances_)
sorted(zip(_feature_importances, _features_names), reverse=True)

In [ ]:
_predictions = random_forest_grid_search.predict(training_features_arr)
_residuals = training_target_arr - _predictions

_fig, _ax = plt.subplots(1, 1, figsize=(10, 7))

_marker_sizes = (training_df.loc[:, "population"]
                            .div(100))

_cmap = plt.get_cmap("viridis")

_kwargs = {'c': "median_house_value",
           's': _marker_sizes,
           "label": "population",
           "alpha": 0.4,
           "cmap": _cmap,
           "ax": _ax}

_ = (training_df.assign(residuals=pd.Series(_residuals, name="residuals"))
                .plot
                .scatter(x="median_income", y="residuals", **_kwargs))

## Evaluate Your System on the Test Set

In [ ]:
testing_df = joblib.load("../data/housing/testing.pkl")

testing_features_arr = preprocessing_pipeline.transform(testing_df)
testing_target_arr = (testing_df.loc[:, "median_house_value"]
                                .values)

In [ ]:
_predictions = (random_forest_grid_search.best_estimator_
                                         .predict(testing_features_arr))

_mse = metrics.mean_squared_error(testing_target_arr, _predictions)
_mse**0.5

In [ ]:
_predictions = (gradient_boosting_randomized_search.best_estimator_
                                                   .predict(testing_features_arr))

_mse = metrics.mean_squared_error(testing_target_arr, _predictions)
_mse**0.5

# Lauch, Monitor, and Maintain Your System